<a href="https://colab.research.google.com/github/Magdalene-Thuo/Week-6_AI-Edge-IoT-Healthcare-2030/blob/EdgeAI_Prototype/edgeAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import os

# Step 1: Create folder structure
base_dir = "/content/dataset"
subdirs = ['train/recyclable', 'train/non_recyclable', 'val/recyclable', 'val/non_recyclable']

for subdir in subdirs:
    os.makedirs(os.path.join(base_dir, subdir), exist_ok=True)

print("Folder structure created.")
from PIL import Image, ImageDraw
import random

def create_image(label, path, count=30):
    for i in range(count):
        img = Image.new('RGB', (64, 64), color='white')
        draw = ImageDraw.Draw(img)

        if label == "recyclable":
            draw.rectangle([10, 10, 54, 54], fill=random.choice(["blue", "green", "cyan"]))
        else:
            draw.ellipse([10, 10, 54, 54], fill=random.choice(["red", "black", "yellow"]))

        img.save(f"{path}/{label}_{i}.png")

# Generate sample images
create_image("recyclable", base_dir + "/train/recyclable")
create_image("non_recyclable", base_dir + "/train/non_recyclable")
create_image("recyclable", base_dir + "/val/recyclable", 10)
create_image("non_recyclable", base_dir + "/val/non_recyclable", 10)

print("✅ Sample images created.")
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set image size and paths
img_height, img_width = 64, 64
batch_size = 16
train_dir = "/content/dataset/train"
val_dir = "/content/dataset/val"

# Normalize images
train_datagen = ImageDataGenerator(rescale=1.0/255)
val_datagen = ImageDataGenerator(rescale=1.0/255)

# Load image batches
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=5, validation_data=val_generator)

# Save it
model.save("/content/recycle_model.h5")
loss, acc = model.evaluate(val_generator)
print(f"✅ Validation Accuracy: {acc:.2f}")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model
with open("recycle_model.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Model converted to TensorFlow Lite.")
import numpy as np
import tensorflow as tf
from PIL import Image
import os

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="recycle_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensor details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load one image to test (adjust the filename if needed)
test_img_path = "/content/dataset/val/recyclable/recyclable_0.png"  # You can change this path
img = Image.open(test_img_path).resize((64, 64))
img_array = np.array(img) / 255.0
img_array = np.expand_dims(img_array.astype(np.float32), axis=0)  # Shape (1, 64, 64, 3)

# Run inference
interpreter.set_tensor(input_details[0]['index'], img_array)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])

# Show result
predicted = "Recyclable" if output[0][0] > 0.5 else "Non-Recyclable"
print(f"🧠 Prediction: {predicted} (Confidence: {output[0][0]:.2f})")







Folder structure created.
✅ Sample images created.
Found 60 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 250ms/step - accuracy: 0.4475 - loss: 0.5988 - val_accuracy: 1.0000 - val_loss: 0.1641
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 1.0000 - loss: 0.1268 - val_accuracy: 1.0000 - val_loss: 0.0255
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 1.0000 - loss: 0.0126 - val_accuracy: 1.0000 - val_loss: 0.0034
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 1.0000 - loss: 0.0021 - val_accuracy: 1.0000 - val_loss: 4.1635e-04
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 1.0000 - loss: 3.1271e-04 - val_accuracy: 1.0000 - val_loss: 8.8501e-05


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 1.0000 - loss: 9.0926e-05
✅ Validation Accuracy: 1.00
Saved artifact at '/tmp/tmpn5618tbh'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name='keras_tensor_32')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  133360568948816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133360568948240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133360568946896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133360568950736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133360568942864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133360568951504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133360568950928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133360568952272: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ Model converted to TensorFlow Lite.
🧠 Prediction:

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
